In [1]:
import numpy as np
from scipy.stats import norm, t
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

A randomized, double-blind experiment was conducted to assess the
effectiveness of several drugs for reducing postoperative nausea. The
data are as follows. (read from file)

(a) Test each drug versus the placebo at the 5 per cent level. Also, report
the estimated odds–ratios. Summarize your findings.
(b) Use the Bonferroni and the FDR method to adjust for multiple
testing. (Beecher (1959)).

In [2]:
data = pd.read_excel('../data/nausea.xlsx', index_col = False, engine = 'openpyxl')
data.index = data['Treatment'].values
data['p^'] = data['Incidence of Nausea'] / data['Number of Patients']
data['se^'] = data['p^'] * (1 - data['p^']) / data['Number of Patients']
p_hat_placebo = data.loc['Placebo', 'p^']
se_hat_placebo = data.loc['Placebo', 'se^']

alpha = 0.05

data['W'] = (data['p^'] - p_hat_placebo) / np.sqrt(data['se^'] + se_hat_placebo)
data['p-value'] = 2 * norm.cdf(-np.abs(data['W']))
data['Rejected Regular'] = data['p-value'] < alpha

bonferroni_threshold = alpha / 4
data['Rejected Bonferroni'] = data['p-value'] < bonferroni_threshold

ordered_pvalues = np.sort(data['p-value'][1:])
Cm, m =1,  len(ordered_pvalues)
l = [i*alpha / (Cm * m) for i in range(1, m+1)]

R = 0
rejected_fdr = [False] # for the placebo
for i in range(m):
    if ordered_pvalues[i] < l[i]:
        R = i+1
        rejected_fdr.append(True)
    else:
        rejected_fdr.append(False)
data['Rejected FDR'] = rejected_fdr
summary = data.iloc[1:]

summary

,Treatment,Number of Patients,Incidence of Nausea,p^,se^,W,p-value,Rejected Regular,Rejected Bonferroni,Rejected FDR
Chlorpromazine,Chlorpromazine,75,26,0.346667,0.003020,-2.764364,0.005703,True,True,True
Dimenhydrinate,Dimenhydrinate,85,52,0.611765,0.002794,0.642987,0.520232,False,False,False
Pentobarbital (100mg),Pentobarbital (100mg),67,35,0.522388,0.003724,-0.486428,0.626664,False,False,False
Pentobarbital (150mg),Pentobarbital (150mg),85,37,0.435294,0.002892,-1.646605,0.099639,False,False,False
